In [26]:
import requests
from bs4 import BeautifulSoup
import json
import matplotlib.pyplot as plt

def get_description(url):
    response = requests.get(url)
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, "html.parser")
        # Find all image tags
        images = soup.find_all("img")
        script = soup.find_all("script", type="application/ld+json")[0]
        description = json.loads(script.string)['description'] 
    else:
        print(f"Failed to fetch the page. HTTP Status Code: {response.status_code}")
    return description

In [27]:
import numpy as np
from io import BytesIO
from PIL import Image
def image_url_to_numpy_array(url, new_width=1080, new_height=1350):
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful
    im = Image.open(BytesIO(response.content))
    width, height = im.size
    aspect_ratio_original = width / height
    aspect_ratio_new = new_width / new_height
    if aspect_ratio_original > aspect_ratio_new:
        scale_factor = new_height / height
    else:
        scale_factor = new_width / width
    target_width = int(width * scale_factor)
    target_height = int(height * scale_factor)
    im = im.resize((target_width, target_height), Image.Resampling.LANCZOS)
    left = (target_width - new_width) / 2
    top = (target_height - new_height) / 2
    right = left + new_width
    bottom = top + new_height
    im = im.crop((left, top, right, bottom))
    img_array = np.array(im)
    return img_array


In [41]:
import requests
import xml.etree.ElementTree as ET
import re

# URL of the RSS feed
url = "https://www.indiatoday.in/rss/home"

# Fetch the RSS feed
response = requests.get(url)

rss = {}

if response.status_code == 200:
    # Parse the XML content
    try:
        root = ET.fromstring(response.content)
        
        # Iterate through the items in the feed
        for idx, item in enumerate(root.findall(".//item")):
            title = item.find("title").text
            link = item.find("link").text[:-1]
            description = item.find("description").text

            rss[idx] = {"Title": title, "Link": link, "Image": re.search("(?P<url>https?://[^\s]+)", description.split("src")[-1]).group("url")[:-2], "Description": get_description(link)}
            
        # print(f"Title: {title}")
        # print(f"Link: {link}")
        # print(f"Image: {re.search("(?P<url>https?://[^\s]+)", description.split("src")[-1]).group("url")}")
    except: pass
else:
    print(f"Failed to fetch the RSS feed. HTTP Status Code: {response.status_code}")

In [42]:
rss

{0: {'Title': "Allu Arjun says 'I respect the law, will cooperate' after leaving jail",
  'Link': ' https://www.indiatoday.in/india/story/allu-arjun-statement-after-released-from-jail-respect-law-abiding-citizen-cooperate-pushpa-2-screening-theatre-stampede-case-2649702-2024-12-14?utm_source=rss',
  'Image': 'https://akm-img-a-in.tosshub.com/indiatoday/images/story/202412/allu-arjun-143413302-16x9_0.png?VersionId=JVf4eubW0rQ7nUtzLtXHMCbRTvOk4QZC',
  'Description': 'Actor Allu Arjun said he would cooperate with authorities in the stampede case after walking out of a Hyderabad jail, where he spent the night following his arrest a day before.'},
 1: {'Title': "No one's asking about stampede victim: Revanth Reddy on Allu Arjun's arrest",
  'Link': ' https://www.indiatoday.in/india/story/telangana-chief-minister-revanth-reddy-on-actor-allu-arjun-arrest-aaj-tak-event-2649539-2024-12-13?utm_source=rss',
  'Image': 'https://akm-img-a-in.tosshub.com/indiatoday/images/story/202412/revanth-reddy-

In [56]:
from PIL import Image, ImageDraw, ImageFont
import textwrap
import random
from time import sleep

for i in range(10):
    rss_choosen = rss[random.randint(0,len(rss)-1)]
    print(rss_choosen)

    fill_image = image_url_to_numpy_array(rss_choosen["Image"])

    template = plt.imread("template.png")
    overlay = Image.open("template.png").convert("RGBA")
    background = Image.fromarray(fill_image)
    background.paste(overlay, (0,0), overlay)

    image = background

    draw = ImageDraw.Draw(image)
    # Assuming `image`, `rss_choosen`, and `draw` are already defined
    font = ImageFont.truetype("DMSans_24pt-ExtraBold.ttf", 50)
    text = rss_choosen["Title"]
    image_width, image_height = image.size
    max_width = image_width - 5
    lines = textwrap.wrap(text, width=45)
    line_height = draw.textsize("A", font=font)[1]
    heading_lines = len(lines)
    y = 1000  # Starting y position
    for line in lines:
        # Calculate text width manually with reduced letter spacing
        letter_spacing = -2  # Reduce letter spacing (negative for tighter spacing)
        text_width = sum(font.getsize(char)[0] for char in line) + letter_spacing * (len(line) - 1)
        text_height = font.getsize(line)[1]

        # Center align each line
        x = (image_width - text_width) // 2

        # Draw a green rectangle as the background for the text
        margin = 4  # Margin around the text
        rectangle_x0 = x
        rectangle_y0 = y
        rectangle_x1 = x + text_width 
        rectangle_y1 = y + text_height 
        draw.rectangle([(rectangle_x0, rectangle_y0), (rectangle_x1, rectangle_y1)], fill=(253, 217, 185))

        # Draw text with reduced letter spacing
        current_x = x
        for char in line:
            draw.text((current_x, y), char, font=font, fill="black")
            current_x += font.getsize(char)[0] + letter_spacing

        # Update y-coordinate for the next line
        y += line_height + margin * 2

    text = rss_choosen["Description"]
    font = ImageFont.truetype("Nunito-VariableFont_wght.ttf", 30)
    # Define the maximum width for the text
    max_width = image_width - 5  # Leave some padding on both sides
    lines = textwrap.wrap(text, width=75)  # Adjust `width` based on text and font size
    line_height = draw.textsize("A", font=font)[1]

    # Start drawing lines
    y = 1100 + (heading_lines-1)*50  # Starting y position
    for line in lines:
        text_width, _ = draw.textsize(line, font=font)
        x = (image_width - text_width) // 2  # Center align each line
        draw.text((x, y), line, font=font, fill="white")
        y += line_height + 5 

    image.save("result.png")

    sleep(3)

{'Title': 'Confident AAP will win with big mandate: Arvind Kejriwal', 'Link': ' https://www.indiatoday.in/elections/assembly/video/confident-aap-will-win-with-big-mandate-arvind-kejriwal-2649600-2024-12-13?utm_source=rss', 'Image': 'https://akm-img-a-in.tosshub.com/indiatoday/images/video/202412/arvind-130846287-16x9_0.jpg?VersionId=.8Sf4tc14WRLqC5.ux_unYXb53X3CIor', 'Description': 'Aam Aadmi Party (AAP) national convenor Arvind Kejriwal said that he is confident that his party will form the government in Delhi with a big mandate.'}


/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:26: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  line_height = draw.textsize("A", font=font)[1]
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:32: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_width = sum(font.getsize(char)[0] for char in line) + letter_spacing * (len(line) - 1)
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:33: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_height = font.getsize(line)[1]
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:50: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength inste

{'Title': 'Watch: Williamson kicks ball onto stumps in bizarre dismissal vs England', 'Link': ' https://www.indiatoday.in/sports/cricket/story/new-zealand-vs-england-kane-williamson-kicks-ball-to-stumps-bizarre-dismissal-matthew-potts-2649721-2024-12-14?utm_source=rss', 'Image': 'https://akm-img-a-in.tosshub.com/indiatoday/images/story/202412/kane-williamson-144813480-16x9_0.jpg?VersionId=aH20VmLD6qIumGhSEYB5OSqgmYwivuCM', 'Description': 'Kane Williamson&#039;s bizarre dismissal, accidentally kicking the ball onto his stumps, marked a turning point as New Zealand&#039;s strong start unravelled against England on Day 1 of the third Test in Hamilton.'}


/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:26: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  line_height = draw.textsize("A", font=font)[1]
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:32: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_width = sum(font.getsize(char)[0] for char in line) + letter_spacing * (len(line) - 1)
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:33: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_height = font.getsize(line)[1]
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:50: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength inste

{'Title': 'AUS vs IND, 3rd Test: Harshit Rana and R Ashwin dropped as India make 2 changes to XI', 'Link': ' https://www.indiatoday.in/sports/story/cricket-border-gavaskar-trophy-3rd-test-harshit-rana-r-ashwin-dropped-ravindra-jadeja-akash-deep-brisbane-2649665-2024-12-14?utm_source=rss', 'Image': 'https://akm-img-a-in.tosshub.com/indiatoday/images/story/202412/aus-vs-ind-140829828-16x9_0.jpg?VersionId=2SVMBQKWCOTqqHw22zVAI.H7g3lRVe8G', 'Description': 'India have decided to drop Harshit Rana and R Ashwin for the Brisbane Test with Ravindra Jadeja and Akash Deep coming into the playing XI. India won the toss on Day 1 of the Test on Saturday, December 14.'}


...

/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:26: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  line_height = draw.textsize("A", font=font)[1]
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:32: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_width = sum(font.getsize(char)[0] for char in line) + letter_spacing * (len(line) - 1)
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:33: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_height = font.getsize(line)[1]
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:50: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength inste

{'Title': 'Proposal for ban on film reviews: Threat to freedom of expression or desperation?', 'Link': ' https://www.indiatoday.in/movies/regional-cinema/story/proposal-for-ban-on-film-reviews-threat-to-freedom-of-expression-or-desperation-cinematic-saturday-2648830-2024-12-14?utm_source=rss', 'Image': 'https://akm-img-a-in.tosshub.com/indiatoday/images/story/202412/ban-film-reviews-134605700-16x9_0.jpg?VersionId=iDHaW.4MMiRhR_Nn2aKvfXyIQ9g.1635', 'Description': 'Recently, the Tamil Film Active Producers Association (TFAPA) filed a writ petition in the Madras High Court calling for a ban on movie reviews on social media for three days since release. Can movie reviews make or break a film&#039;s theatrical run?'}


/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:26: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  line_height = draw.textsize("A", font=font)[1]
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:32: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_width = sum(font.getsize(char)[0] for char in line) + letter_spacing * (len(line) - 1)
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:33: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_height = font.getsize(line)[1]
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:50: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength inste

{'Title': "In China, people hiring 'climbing buddies' for treks is the latest trend", 'Link': ' https://www.indiatoday.in/trending-news/story/china-latest-trend-people-hiring-climbing-buddies-for-trek-2649725-2024-12-14?utm_source=rss', 'Image': 'https://akm-img-a-in.tosshub.com/indiatoday/images/story/202412/chinas-latest-trend-hiring-climbing-buddies-for-mountain-treks-140601879-16x9_0.jpg?VersionId=ln2EeX6TcdXMBwdhN2Pp6.Zm9GZAorqz', 'Description': 'Hiring &quot;climbing buddies&quot; has become the latest trend in China, with people paying for companions to guide and motivate them on mountain treks. Read on to know more.'}


/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:26: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  line_height = draw.textsize("A", font=font)[1]
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:32: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_width = sum(font.getsize(char)[0] for char in line) + letter_spacing * (len(line) - 1)
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:33: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  text_height = font.getsize(line)[1]
/var/folders/3p/xlr6tgyx4t980qpxnnrs12kc0000gn/T/ipykernel_6177/4043857838.py:50: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength inste